<a href="https://colab.research.google.com/github/sokrypton/7.571/blob/main/L2/CLT_Bootstrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Central Limit Theorem & Bootstrap

**Goals:**
1. Understand the Central Limit Theorem (CLT) - a closed-form solution to estimate standard error
2. Understand Bootstrap - a computational approach to estimate standard error

---

In [ ]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt

# The Problem: Comparing Two Populations

Imagine you're comparing gene expression between control and treatment groups.

You measure samples from each group and calculate the means. But how do you know if the difference is **real** or just **sampling noise**?

In [ ]:
# Two populations we want to compare
mu_control = 10.0
mu_treatment = 12.0
sigma = 4.0  # same variability in both

# Take samples from each
n = 50
control_sample = np.random.normal(loc=mu_control, scale=sigma, size=n)
treatment_sample = np.random.normal(loc=mu_treatment, scale=sigma, size=n)

# Calculate means
control_mean = np.mean(control_sample)
treatment_mean = np.mean(treatment_sample)

print(f"Control mean:   {control_mean:.2f}")
print(f"Treatment mean: {treatment_mean:.2f}")
print(f"Difference:     {treatment_mean - control_mean:.2f}")
print()
print("But wait... how do we know this difference is real?")
print("What if we just got unlucky with our samples?")

## This is why we need Standard Error!

**Standard Error (SE)** tells us how much our estimate might vary if we took a different sample.

Formula: **SE = s / sqrt(n)**

In [ ]:
# Calculate SE for each group
control_SE = np.std(control_sample) / np.sqrt(n)
treatment_SE = np.std(treatment_sample) / np.sqrt(n)

print(f"Control:   {control_mean:.2f} +/- {control_SE:.2f}")
print(f"Treatment: {treatment_mean:.2f} +/- {treatment_SE:.2f}")
print()
print("Now we can see how confident we are in each estimate!")

## But where does this formula come from?

**The Central Limit Theorem** tells us that SE = sigma / sqrt(n)

Let's prove it!

---

# Part 1: Central Limit Theorem (CLT)

## Normal Distribution

In [ ]:
# Define population parameters
mu = 2.0      # population mean
sigma = 4.0   # population std

# Take ONE sample
n = 100
samples = np.random.normal(loc=mu, scale=sigma, size=n)
plt.hist(samples, bins=10)
plt.title('One Sample from Normal Distribution')
plt.show()

In [ ]:
# Sample mean (our estimate of mu)
x_hat = np.mean(samples)
print(f"Sample mean: {x_hat:.4f}")

In [ ]:
# Estimated standard error using CLT formula: SE = s / sqrt(n)
SE = np.std(samples) / np.sqrt(n)
print(f"Standard Error: {SE:.4f}")

### Let's verify CLT by repeating the experiment many times

In [ ]:
# Repeat the experiment many times
num_experiments = 1000
sample_means = []

for i in range(num_experiments):
    samples = np.random.normal(loc=mu, scale=sigma, size=n)
    x_hat = np.mean(samples)
    sample_means.append(x_hat)

sample_means = np.array(sample_means)

In [ ]:
# Plot the distribution of sample means
plt.hist(sample_means, bins=30, edgecolor='white')
plt.axvline(mu, color='red', linestyle='--', label=f'mu = {mu}')

# Add std lines
theoretical_se = sigma / np.sqrt(n)
plt.axvline(mu - theoretical_se, color='orange', linestyle='--', label='mu +/- sigma/sqrt(n)')
plt.axvline(mu + theoretical_se, color='orange', linestyle='--')

plt.xlabel('Sample Mean')
plt.title('Distribution of Sample Means')
plt.legend()
plt.show()

print("Verifying CLT")
print("=" * 40)
print(f"Observed std of sample means:  {np.std(sample_means):.4f}")
print(f"Theoretical sigma/sqrt(n):     {sigma/np.sqrt(n):.4f}")

## Exponential Distribution

CLT works for ANY distribution - let's try one that is clearly NOT normal!

In [ ]:
# Let's try a different distribution
lam = 0.5
n = 100
samples = np.random.exponential(scale=1/lam, size=n)
plt.hist(samples, bins=10)
plt.title('Exponential Distribution (NOT normal!)')
plt.show()

In [ ]:
# Repeat the experiment many times
num_experiments = 1000
sample_means = []

for i in range(num_experiments):
    samples = np.random.exponential(scale=1/lam, size=n)
    sample_means.append(np.mean(samples))

sample_means = np.array(sample_means)

In [ ]:
# For exponential with rate lambda: mu = 1/lambda, sigma = 1/lambda
mu = 1/lam
sigma = 1/lam

# Plot the distribution of sample means
plt.hist(sample_means, bins=30, edgecolor='white')
plt.axvline(mu, color='red', linestyle='--', label=f'mu = {mu}')

# Add std lines
theoretical_se = sigma / np.sqrt(n)
plt.axvline(mu - theoretical_se, color='orange', linestyle='--', label='mu +/- sigma/sqrt(n)')
plt.axvline(mu + theoretical_se, color='orange', linestyle='--')

plt.xlabel('Sample Mean')
plt.title('Distribution of Sample Means (normal!)')
plt.legend()
plt.show()

print("Verifying CLT")
print("=" * 40)
print(f"Observed std of sample means:  {np.std(sample_means):.4f}")
print(f"Theoretical sigma/sqrt(n):     {sigma/np.sqrt(n):.4f}")

## So What Does This Mean?

We did 1000 experiments to prove CLT works.

**But in real research, you only get ONE sample!**

CLT tells us:
- We can estimate standard error from just one sample using: `SE = s / sqrt(n)`
- This tells us how much to trust our estimate of the mean

**But what if we want SE of something other than the mean?**

---
# Part 2: Bootstrap

## Step 1: The Problem

In [ ]:
# In real life, you only have ONE sample
# And you DON'T know the true mu or sigma

lam = 0.5  # In real life, you wouldn't know this!
n = 100
my_sample = np.random.exponential(scale=1/lam, size=n)

print("In real life, all you have is your sample:")
print(f"Sample mean: {np.mean(my_sample):.3f}")
print(f"Sample std:  {np.std(my_sample):.3f}")
print(f"Sample size: {n}")

## Step 2: CLT Approach Works for the Mean

In [ ]:
# We can estimate SE using the CLT formula
SE = np.std(my_sample) / np.sqrt(n)
print(f"SE of mean (using formula): {SE:.4f}")

## Step 3: But What About Other Statistics?

In [ ]:
# What if we want SE of the median?
sample_median = np.median(my_sample)
print(f"Sample median: {sample_median:.3f}")
print(f"SE of median:  ???")  # No formula!

## Step 4: The Bootstrap Idea

Key insight: **Your sample is your best estimate of the population.**

So... what if we resample FROM our sample?

In [ ]:
# This is called resampling WITH REPLACEMENT
resample = np.random.choice(my_sample, size=n, replace=True)

print("Original sample (first 10):", my_sample[:10].round(2))
print("One resample (first 10):   ", resample[:10].round(2))
print("\nNotice: some values repeat, some are missing!")

## Step 5: Do It Many Times

In [ ]:
# Resample many times, calculate mean each time
num_resamples = 1000
bootstrap_means = []

for i in range(num_resamples):
    resample = np.random.choice(my_sample, size=n, replace=True)
    bootstrap_means.append(np.mean(resample))

bootstrap_means = np.array(bootstrap_means)

## Step 6: The Bootstrap SE

In [ ]:
# The std of bootstrap means = SE of the mean!
bootstrap_SE = np.std(bootstrap_means)
formula_SE = np.std(my_sample) / np.sqrt(n)

print("SE of the Mean")
print("=" * 40)
print(f"Bootstrap SE:        {bootstrap_SE:.4f}")
print(f"Formula SE (s/sqrt(n)): {formula_SE:.4f}")
print("\nThey match! Bootstrap works.")

## Step 7: Visualize It

In [ ]:
plt.hist(bootstrap_means, bins=30, edgecolor='white')
plt.axvline(np.mean(my_sample), color='red', linestyle='--', label='Sample mean')
plt.xlabel('Bootstrap Sample Mean')
plt.title('Bootstrap Distribution of the Mean')
plt.legend()
plt.show()

## Step 8: Now the Magic - SE of the Median!

No formula exists, but bootstrap doesn't care!

In [ ]:
bootstrap_medians = []

for i in range(num_resamples):
    resample = np.random.choice(my_sample, size=n, replace=True)
    bootstrap_medians.append(np.median(resample))

bootstrap_medians = np.array(bootstrap_medians)

print("SE of the Median")
print("=" * 40)
print(f"Sample median:  {np.median(my_sample):.4f}")
print(f"Bootstrap SE:   {np.std(bootstrap_medians):.4f}")
print("\nNo formula needed!")

## Step 9: Prove Bootstrap Gives Correct SE for Median

To verify bootstrap works, let's compare to the "true" SE (from many experiments).

In [ ]:
# Do the real experiment many times (just like we did for CLT)
num_experiments = 1000
sample_medians = []

for i in range(num_experiments):
    samples = np.random.exponential(scale=1/lam, size=n)
    sample_medians.append(np.median(samples))

sample_medians = np.array(sample_medians)
true_SE_median = np.std(sample_medians)

print("Verifying Bootstrap for the Median")
print("=" * 45)
print(f"True SE (from 1000 experiments):  {true_SE_median:.4f}")
print(f"Bootstrap SE (from ONE sample):   {np.std(bootstrap_medians):.4f}")
print()
print("Bootstrap estimates the SE using only ONE sample!")

---
# Summary

## Central Limit Theorem (CLT)
- **What:** Distribution of sample means approaches normal, regardless of population shape
- **Formula:** SE = sigma / sqrt(n)  (estimate with s / sqrt(n))
- **Limitation:** Only works for the mean

## Bootstrap
- **What:** Computational method to estimate SE of ANY statistic
- **How:** Resample with replacement -> Calculate statistic -> Repeat many times -> Take std
- **Advantage:** Works for mean, median, or any statistic you can compute!

| Method | SE Formula | Works For |
|--------|------------|----------|
| CLT | s / sqrt(n) | Mean only |
| Bootstrap | std(bootstrap statistics) | Anything! |